In [1]:
import numpy as np 
import random 

In [143]:
def generate_sum_delete_instance(k, deletion_rate=1/3,min_num=1,max_num=10):
    
    matrix = np.random.randint(min_num, max_num, (k, k))

    flat_matrix = matrix.flatten()
    num_deletions = int(len(flat_matrix) * deletion_rate)
    indexes_to_delete = random.sample(range(len(flat_matrix)), num_deletions)
    for index in indexes_to_delete:
        flat_matrix[index] = 0  

    # calculating the sums after delettion
    deleted_matrix = flat_matrix.reshape(k, k)
    target_row_sums = np.sum(deleted_matrix, axis=1)
    target_col_sums = np.sum(deleted_matrix, axis=0)
    
    # random values for deleted numbers
    for i in range(len(deleted_matrix)):
        for j in range(len(deleted_matrix[0])):
            if deleted_matrix[i][j] == 0:
                deleted_matrix[i][j] = np.random.randint(min_num,max_num)
    

    return deleted_matrix, target_row_sums, target_col_sums

######################################################
k = 6  
instance, row_sums, col_sums = generate_sum_delete_instance(k)
print( instance)
print("target rows" ,row_sums)
print("target_cols", col_sums)


[[3 6 5 3 9 7]
 [2 6 9 8 1 4]
 [3 6 4 6 1 9]
 [6 9 9 7 8 9]
 [2 3 7 7 6 5]
 [8 1 9 2 5 3]]
target rows [23 15 14 48 18  7]
target_cols [16 18 25 23 30 13]


In [89]:
########### FITNESS FUNCITONS #############
    
def calculate_fitness(self,chromosome): # Fitness is higher when the sum difference is lower
    current_row_sums = np.sum(chromosome * self.instance, axis=1)
    current_col_sums = np.sum(chromosome * self.instance, axis=0)
    row_diff = np.sum(np.abs(self.target_row_sums - current_row_sums))
    col_diff = np.sum(np.abs(self.target_col_sums - current_col_sums))
    return 1 / (1 + row_diff + col_diff)  

def fitness_a(self,chromosome): # 1 for correct and 0 for all others
    
    current_row_sums = np.sum(chromosome * self.instance, axis=1)
    current_col_sums = np.sum(chromosome * self.instance, axis=0)
    if np.array_equal(current_row_sums, self.target_row_sums) and np.array_equal(current_col_sums, self.target_col_sums):
        return 1
    else:
        return 0


In [36]:
########### SELECTION FUNCTIONS ##################

def tournament_selection(self, tournament_size):
        """
        Selects parents using tournament selection.

        :param tournament_size: The number of individuals participating in each tournament.
        :return: List of selected parents
        """
        selected_parents = []
        for _ in range(len(self.population)):
            # Randomly select tournament participants
            participants = random.sample(self.population, tournament_size)
            # Evaluate the fitness of tournament participants
            participant_fitnesses = [self.calculate_fitness(individual) for individual in participants]
            # Select the best individual from participants
            winner_index = np.argmax(participant_fitnesses)
            selected_parents.append(participants[winner_index])
        
        return selected_parents

def practical_roulette_selection(population, fitness_scores):
    average_fitness = np.mean(fitness_scores)
    new_population = []
    
    for i, individual in enumerate(population):
        fitness = fitness_scores[i]
        
        if fitness == average_fitness:
            # The individual survives
            new_population.append(individual)
        elif fitness < average_fitness:
            # The individual survives with a probability of f_i / f̄
            if random.random() < (fitness / average_fitness):
                new_population.append(individual)
        else:
            # The individual produces floor(f_i / f̄) offspring
            offspring_count = int(fitness // average_fitness)
            new_population.extend([individual] * offspring_count)
            
            # The individual has a chance to produce one more offspring
            if random.random() < (fitness / average_fitness - offspring_count):
                new_population.append(individual)
    
    return new_population

In [151]:
####################### CROSSOVER METHODS #############################
def single_point_crossover(parent1, parent2, p_c):
    # Only perform crossover with probability p_c
    if np.random.rand() < p_c:
        # Choose a random crossover point
        crossover_point = np.random.randint(1, len(parent1))  # start from 1 to avoid null crossover
        # Create children by combining the parents at the crossover point
        child1 = np.vstack((parent1[:crossover_point, :], parent2[crossover_point:, :]))
        child2 = np.vstack((parent2[:crossover_point, :], parent1[crossover_point:, :]))
    else:
        # With probability 1 - p_c, copy the parents
        child1, child2 = np.copy(parent1), np.copy(parent2)

    return child1, child2


In [14]:
####################### MUTATION FUNCTIONS ####################
def mutate_bit_flip(chromosome, mutation_rate):
    mutation_matrix = np.random.rand(len(chromosome),len(chromosome)) < mutation_rate
    # XOR with mutation matrix will flip the bits where mutation_matrix is True
    chromosome = np.logical_xor(chromosome, mutation_matrix).astype(int)
    return chromosome


def mutate_random_resetting(chromosome, mutation_rate):
    for i in range(len(chromosome)):
        for j in range(self.k):
            if np.random.rand() < mutation_rate:
                chromosome[i, j] = 1 if chromosome[i, j] == 0 else 0
    return chromosome


In [142]:
########################### OPTIMALITY VERIFICATION ##################

def verify_solution(instance, solution, target_row_sums, target_col_sums):
    """
    Verifies if the given solution is the best by comparing the row and column sums
    with the target row and column sums.

    :param instance: The original k-by-k matrix of the Sum Delete problem.
    :param solution: A k-by-k binary matrix representing the solution to verify.
    :param target_row_sums: Array of target sums for each row.
    :param target_col_sums: Array of target sums for each column.
    :return: True if the solution is optimal, False otherwise.
    """
    # PRETTY PRINT 
    print("Given Instance : \n" , instance)
    print("target rows sums :", target_row_sums)
    print("target column sums :", target_col_sums)
    
    
    
    # Calculate the sum of the rows and columns in the solution
    solution_row_sums = np.sum(solution * instance, axis=1)
    print("Row Sums after deletion",solution_row_sums)
    solution_col_sums = np.sum(solution * instance, axis=0)
    print("Column Sums after deletion",solution_col_sums)
    

    # Check if the calculated sums match the target sums
    is_row_correct = np.array_equal(solution_row_sums, target_row_sums)
    is_col_correct = np.array_equal(solution_col_sums, target_col_sums)

    # The solution is optimal if both row and column sums match the target sums
    print("is the solution optimal?? :",is_row_correct and is_col_correct)
    return is_row_correct and is_col_correct


In [212]:
#### GENETIC ALGORITHM SOLVER

class GA:
    
    def __init__(self,population_size,instance,target_row_sums,target_col_sums,p_c,p_m):
        self.k = len(instance)
        self.instance = instance
        self.target_row_sums = target_row_sums
        self.target_col_sums = target_col_sums
        self.population = self.generate_initial_population(population_size,self.k)
        
        self.p_c = p_c
        self.p_m = p_m
        self.elitism_size = 5
    
    def generate_initial_population(self,population_size, k):
        population = []
        for _ in range(population_size):
            chromosome = np.random.choice([0, 1], (k, k))
            population.append(chromosome)
        return population
    
    def calculate_fitness(self,chromosome): # Fitness is higher when the sum difference is lower
        current_row_sums = np.sum(chromosome * self.instance, axis=1)
        current_col_sums = np.sum(chromosome * self.instance, axis=0)
        row_diff = np.sum(np.abs(self.target_row_sums - current_row_sums))
        col_diff = np.sum(np.abs(self.target_col_sums - current_col_sums))
        return 1 / (1 + row_diff + col_diff)  
            
    def select_parents(self,population, fitness_scores):
        return self.tournament_selection()

    def tournament_selection(self, tournament_size=3):
        
        selected_parents = []
        for _ in range(len(self.population)):
            # Randomly select tournament participants
            participants = random.sample(self.population, tournament_size)
            # Evaluate the fitness of tournament participants
            participant_fitnesses = [self.calculate_fitness(individual) for individual in participants]
            # Select the best individual from participants
            winner_index = np.argmax(participant_fitnesses)
            selected_parents.append(participants[winner_index])
        
        return selected_parents

    def crossover(self,parent1, parent2):
        return single_point_crossover(parent1, parent2, self.p_c)

    def mutate(self, chromosome): ### _bit_flip
        mutation_matrix = np.random.rand(self.k, self.k) < self.p_m
        # XOR with mutation matrix will flip the bits where mutation_matrix is True
        chromosome = np.logical_xor(chromosome, mutation_matrix).astype(int)
        return chromosome
    
    def get_elite_group(self,fitness_values):
        sorted_population = [x for _,x in sorted(zip(fitness_values,self.population),key=lambda pair:pair[0], reverse=True)]
        
        return sorted_population[:self.elitism_size]


    def solve(self, generations):
        best_solution = None
        best_fitness = 0

        for generation in range(generations):
        
            fitness_scores = [self.calculate_fitness(chromosome) for chromosome in self.population]
            
            # Check for the best solution
            current_best = max(fitness_scores)
            if current_best > best_fitness:
                best_fitness = current_best
                best_solution = self.population[np.argmax(fitness_scores)]

            # Termination condition
            if best_fitness == 1:  # Perfect fitness
                break
            
            # Selection
            parents = self.select_parents(self.population, fitness_scores)
            # print("number of parents", len(parents))
            
            # Crossover and mutation
            new_population = self.get_elite_group(fitness_scores)
            for _ in range(len(parents) // 2):
                parent1, parent2 = random.sample(parents, 2)
                child1, child2 = self.crossover(parent1, parent2)
                child1 = self.mutate(child1)
                child2 = self.mutate(child2)
                new_population.extend([child1, child2])
            
            # Replacement
            # print("len of new population", len(new_population))
            self.population = new_population[:len(self.population)]
            
            

        return best_solution

# DRIVER CODE 
k = 6 # Dimension of the matrix
population_size = 100
generations = 1000
mutation_rate = 0.01
crossover_rate = 0.7
instance, row_sums, col_sums = generate_sum_delete_instance(k)
algorithm = GA(population_size,instance,row_sums,col_sums,crossover_rate,mutation_rate)
solution = algorithm.solve(generations)


In [213]:
verify_solution(instance, solution, row_sums, col_sums)

Given Instance : 
 [[9 5 7 4 4 3]
 [9 4 7 8 6 7]
 [3 4 8 3 5 7]
 [9 4 8 8 5 6]
 [6 1 3 6 8 5]
 [7 4 8 6 3 8]]
target rows sums : [ 5 31 19 34 23 28]
target column sums : [31 18 18 25 21 27]
Row Sums after deletion [ 5 28 19 36 23 28]
Column Sums after deletion [31 18 18 25 21 26]
is the solution optimal?? : False


False

In [165]:
import pygad
import numpy as np

# Define the fitness function
def fitness_func(ga_instance ,solution, solution_idx):
    global target_row_sums, target_col_sums, instance_matrix

    # Reshape the solution back into a matrix to calculate row/col sums
    solution_matrix = np.reshape(solution, (k, k))
    
    # Calculate the sum of the rows and columns in the solution
    current_row_sums = np.sum(solution_matrix * instance_matrix, axis=1)
    current_col_sums = np.sum(solution_matrix * instance_matrix, axis=0)
    
    # Calculate the fitness score as the inverse of the sum of absolute differences
    fitness = 1.0 / (np.sum(np.abs(target_row_sums - current_row_sums)) + 
                     np.sum(np.abs(target_col_sums - current_col_sums)) + 1.0)
    return fitness

# Define other GA parameters
k = 3  # The dimension of the matrix
instance_matrix, target_row_sums, target_col_sums = generate_sum_delete_instance(k)
population_size = 100
num_generations = 1000
mutation_probability = 0.1  # Mutation rate
crossover_probability = 0.8  # Crossover rate

# Initialize the GA
ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=4,
                       fitness_func=fitness_func,
                       sol_per_pop=population_size,
                       num_genes=k*k,
                       init_range_low=0,
                       init_range_high=2,
                       gene_type=int,
                       parent_selection_type="sss",
                       crossover_type="two_points",
                       crossover_probability=crossover_probability,
                       mutation_type="inversion",
                       mutation_probability=mutation_probability,
                       
                       )

# Run the GA
ga_instance.run()

# Fetch the best solution
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Parameters of the best solution : {solution}".format(solution=solution))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))

# Reshape the solution to the matrix form to display it
solution_matrix = np.reshape(solution, (k, k))
print("Best solution as a matrix:\n", solution_matrix)


Parameters of the best solution : [1 0 1 0 1 1 1 0 1]
Fitness value of the best solution = 1.0
Best solution as a matrix:
 [[1 0 1]
 [0 1 1]
 [1 0 1]]
